In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
leases = gpd.read_parquet("our_data/leases_with_nearest_quadrangles.parquet")
leases.info()

In [ ]:
from shapely import wkt
wells = pd.read_parquet("our_data/wells_with_depth_trend.parquet")
wells['geometry'] = gpd.points_from_xy(
    x=wells.SITE_LONGIT_DECIMAL, 
    y=wells.SITE_LATITUDE_DECIMAL, 
    crs="EPSG:4326"
    )
wells = gpd.GeoDataFrame(data=wells, geometry='geometry')
wells.info()

In [ ]:
wells_on_lease = gpd.sjoin(left_df=leases, right_df=wells, how='left', predicate="contains")
wells_on_lease = wells_on_lease.groupby("lease_").agg({
    'depth_20_year_trend':'mean',
    })
leases_with_depth_trend = leases.merge(wells_on_lease, on="lease_", suffixes = ["", "_y"])
leases_with_depth_trend.info()
# leases = gpd.join(left_df=leases, right_df = joined_data, on="lease_")

In [ ]:
leases = leases.set_geometry("nearest_quadrangle").set_crs(crs=3857)
leases = leases.set_geometry("nearest_quadrangle").to_crs(crs=4326)
# leases = leases.set_geometry("nearest_quadrangle").set_crs("EPSG:4326")
wells_on_nearest_quadrangle = gpd.sjoin(left_df=leases, right_df=wells, how='left', predicate="contains")
wells_on_nearest_quadrangle = wells_on_nearest_quadrangle.groupby("lease_").agg({
    'depth_20_year_trend':'mean',
    })
wells_on_nearest_quadrangle = wells_on_nearest_quadrangle.rename(columns ={'depth_20_year_trend':'nearby_depth_20_year_trend'})
wells_on_nearest_quadrangle.info()
leases_with_depth_trend = leases_with_depth_trend.merge(wells_on_nearest_quadrangle, on="lease_", suffixes=["", "_x"])

In [ ]:

leases = leases.set_geometry("lease_quadrangle").set_crs(crs=3857)
leases = leases.set_geometry("lease_quadrangle").to_crs(crs=4326)
# leases = leases.drop(['index_right'], axis=1)

wells_on_lease_quadrangle = gpd.sjoin(left_df=leases, right_df=wells, how='left', predicate="contains")
wells_on_lease_quadrangle = wells_on_lease_quadrangle.groupby("lease_").agg({
    'depth_20_year_trend':'mean',
    })
wells_on_lease_quadrangle = wells_on_lease_quadrangle.rename(columns ={'depth_20_year_trend':'lease_quadrangle_depth_20_year_trend'})
leases_with_depth_trend = leases_with_depth_trend.merge(wells_on_lease_quadrangle, on="lease_", suffixes=["", "_x"])

In [ ]:
leases_with_depth_trend.info()

In [ ]:
print(leases_with_depth_trend["depth_20_year_trend"].mean())
print(leases_with_depth_trend["lease_quadrangle_depth_20_year_trend"].mean())
print(leases_with_depth_trend["nearby_depth_20_year_trend"].mean())


In [ ]:
leases_with_depth_trend.to_parquet("our_data/processed_data.parquet")

In [ ]:
leases_with_depth_trend[[
    "depth_20_year_trend", 
    "lease_quadrangle_depth_20_year_trend", 
    "nearby_depth_20_year_trend"]].info()

In [ ]:
# the geometries you have are the nearest_quardrangle, lease_quadrangle, and geometry (the lease)
# random change